In [2]:
!pip install xlsxwriter 
import pandas as pd
import xlsxwriter


  Obtaining dependency information for xlsxwriter from https://files.pythonhosted.org/packages/a7/ea/53d1fe468e63e092cf16e2c18d16f50c29851242f9dd12d6a66e0d7f0d02/XlsxWriter-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/159.9 kB ? eta -:--:--
   ------- -------------------------------- 30.7/159.9 kB 1.3 MB/s eta 0:00:01
   -------------- ------------------------ 61.4/159.9 kB 656.4 kB/s eta 0:00:01
   ----------------------------- -------- 122.9/159.9 kB 717.5 kB/s eta 0:00:01
   -------------------------------------- 159.9/159.9 kB 953.4 kB/s eta 0:00:00


In [3]:
excel_file = 'Datesheet Generator.xlsx'

In [4]:

df = pd.read_excel(excel_file)


In [5]:
df

,S.No,Program,Semester,Course Code,Course Name,Course Type,No. of Students,L-T-P
0,1,B.Sc Chemistry,IV,CHH205B-T,PHYSICAL CHEMISTRY III,Core,9.0,3-1-0
1,2,B.Sc Chemistry,IV,CHH206B-T,INORGANIC CHEMISTRY III,Core,9.0,3-1-0
2,3,B.Sc Chemistry,IV,CHH207B-T,ORGANIC CHEMISTRY III,Core,9.0,0-0-3
3,4,B.Sc Chemistry,IV,CHH209B,INDUSTRIAL CHEMISTRY,Elective,9.0,3-1-0
4,5,B.Sc Chemistry,VI,CHH310B,Chemicals & Environment,Elective,5.0,3-1-0
...,...,...,...,...,...,...,...,...
478,479,B.Tech CSE - CSTI,VI,CSH313B-T,DATA WAREHOUSE & DATA MINING,Elective,30.0,3-0-2
479,480,B.Tech CSE - CSTI,VI,CSH314B,MACHINE LEARNING,Elective,30.0,3-0-2
480,481,B.Tech CSE - CSTI,VI,CSH306B-T,SYSTEM NETWORK & ADMINISTRATION,Elective,30.0,3-0-2
481,482,B.Tech CSE - CSTI,VI,LWS323/LWS325,CYBER LAW/\nLAW RELATING TO INTELLECTUAL PROPE...,Elective,30.0,2-0-0


In [13]:
df2 = pd.DataFrame()

In [14]:
df2["S.No"] = df["S.No"]

In [15]:
df2["Program"] = df["Program"]

In [16]:
df2["Semester"] = df["Semester"]

In [17]:
df2["Course Code"] = df["Course Code"]

In [18]:
df2["Course Name"] = df["Course Name"]

In [19]:
df2

,S.No,Program,Semester,Course Code,Course Name
0,1,B.Sc Chemistry,IV,CHH205B-T,PHYSICAL CHEMISTRY III
1,2,B.Sc Chemistry,IV,CHH206B-T,INORGANIC CHEMISTRY III
2,3,B.Sc Chemistry,IV,CHH207B-T,ORGANIC CHEMISTRY III
3,4,B.Sc Chemistry,IV,CHH209B,INDUSTRIAL CHEMISTRY
4,5,B.Sc Chemistry,VI,CHH310B,Chemicals & Environment
...,...,...,...,...,...
478,479,B.Tech CSE - CSTI,VI,CSH313B-T,DATA WAREHOUSE & DATA MINING
479,480,B.Tech CSE - CSTI,VI,CSH314B,MACHINE LEARNING
480,481,B.Tech CSE - CSTI,VI,CSH306B-T,SYSTEM NETWORK & ADMINISTRATION
481,482,B.Tech CSE - CSTI,VI,LWS323/LWS325,CYBER LAW/\nLAW RELATING TO INTELLECTUAL PROPE...


In [46]:
import numpy as np
import pandas as pd

# Assuming df2 is your DataFrame containing the data
df2[['L', 'T', 'P']] = df['L-T-P'].str.split('-', expand=True)
df2[['L', 'T', 'P']] = df2[['L', 'T', 'P']].apply(pd.to_numeric)
df2 = df2[(df2['L'] != 0) | (df2['T'] != 0)]

start_date = '2024-03-11'
end_date = '2024-04-28'
date_range = pd.date_range(start=start_date, end=end_date)
weekdays_range = date_range[date_range.dayofweek < 5]

# Define time slots
time_slots = [
    '9:00 - 10:30'
    
]

date_index = 0
time_slot_index = 0 
last_date = {} 
prev_program_semester = None  

for i, row in df2.iterrows():
    program_semester = (row["Program"], row["Semester"]) 
    
    if pd.isna(row["Course Name"]):  
        df2.at[i, 'Date'] = np.nan  
        date_index = 0  
    else:
        if program_semester != prev_program_semester:
            date_index = 0  
            prev_program_semester = program_semester
            time_slot_index = 0  # Reset the time slot index when program/semester changes
        
        if program_semester in last_date:
            last_assigned_date = last_date[program_semester] + pd.Timedelta(days=2) 
        else:
            last_assigned_date = pd.Timestamp(weekdays_range[0])  
        
        while True:
            if date_index >= len(weekdays_range):
                df2.at[i, 'Date'] = np.nan  
                break
            elif weekdays_range[date_index].dayofweek < 5:
                current_date = pd.Timestamp(weekdays_range[date_index].date())  
                if current_date >= last_assigned_date:
                    df2.at[i, 'Date'] = current_date  
                    last_date[program_semester] = current_date 
                    
                    # Assign time slot
                    df2.at[i, 'Time Slot'] = time_slots[time_slot_index]
                    
                    time_slot_index = (time_slot_index + 1) % len(time_slots)
                    
                    date_index += 1  
                    break
                else:
                    date_index += 1  
            else:
                date_index += 1  
                time_slot_index = 2 if time_slot_index == 0 else 0  


In [47]:
df2

,S.No,Program,Semester,Course Code,Course Name,Date,L,T,P,Time Slot
0,1,B.Sc Chemistry,IV,CHH205B-T,PHYSICAL CHEMISTRY III,2024-03-11,3.0,1.0,0.0,9:00 - 10:30
1,2,B.Sc Chemistry,IV,CHH206B-T,INORGANIC CHEMISTRY III,2024-03-13,3.0,1.0,0.0,9:00 - 10:30
3,4,B.Sc Chemistry,IV,CHH209B,INDUSTRIAL CHEMISTRY,2024-03-15,3.0,1.0,0.0,9:00 - 10:30
4,5,B.Sc Chemistry,VI,CHH310B,Chemicals & Environment,2024-03-11,3.0,1.0,0.0,9:00 - 10:30
5,6,M.Sc Chemistry,II,CHH506B,Physical Chemistry-II,2024-03-11,4.0,0.0,0.0,9:00 - 10:30
...,...,...,...,...,...,...,...,...,...,...
478,479,B.Tech CSE - CSTI,VI,CSH313B-T,DATA WAREHOUSE & DATA MINING,2024-03-22,3.0,0.0,2.0,9:00 - 10:30
479,480,B.Tech CSE - CSTI,VI,CSH314B,MACHINE LEARNING,2024-03-25,3.0,0.0,2.0,9:00 - 10:30
480,481,B.Tech CSE - CSTI,VI,CSH306B-T,SYSTEM NETWORK & ADMINISTRATION,2024-03-27,3.0,0.0,2.0,9:00 - 10:30
481,482,B.Tech CSE - CSTI,VI,LWS323/LWS325,CYBER LAW/\nLAW RELATING TO INTELLECTUAL PROPE...,2024-03-29,2.0,0.0,0.0,9:00 - 10:30


In [48]:

excel_file_path = 'output_data.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    df2.to_excel(writer, index=False)
    worksheet = writer.sheets['Sheet1'] 
    worksheet.set_column('E:E', 6) 
